In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from diffusers import DDPMScheduler, UNet2DModel, DDPMPipeline
from diffusers.optimization import get_cosine_schedule_with_warmup
from diffusers.models.embeddings import Timesteps, TimestepEmbedding
from diffusers.models.unets.unet_2d_blocks import UNetMidBlock2D, get_down_block, get_up_block
from dataclasses import dataclass
from PIL import Image
import os
import numpy as np
from tqdm.auto import tqdm

In [12]:
KOOPMAN = False

In [13]:
@dataclass
class TrainingConfig:
    # Optimization
    image_size = 32
    train_batch_size = 64
    eval_batch_size = 64
    num_epochs = 20
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 5
    save_model_epochs = 10
    seed = 0

    # Dynamic Output Directory
    output_dir = "ddpm_mnist_koopman" if KOOPMAN else "ddpm_mnist_baseline"

    # Shared Architecture (Used by BOTH models for fair comparison)
    in_channels = 1
    out_channels = 1
    block_out_channels = (32, 64, 128, 128)
    layers_per_block = 2
    down_block_types = ("DownBlock2D", "DownBlock2D", "AttnDownBlock2D", "DownBlock2D")
    up_block_types = ("UpBlock2D", "AttnUpBlock2D", "UpBlock2D", "UpBlock2D")
    sample_size = image_size

config = TrainingConfig()
os.makedirs(config.output_dir, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
def get_dataloader(config):
    preprocess = transforms.Compose([
        transforms.Resize((config.image_size, config.image_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ])
    dataset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=preprocess)
    return DataLoader(dataset, batch_size=config.train_batch_size, shuffle=True)

def save_sample(model, scheduler, epoch, config):
    model.eval()
    pipeline = DDPMPipeline(unet=model, scheduler=scheduler)
    images = pipeline(batch_size=config.eval_batch_size, generator=torch.manual_seed(config.seed), num_inference_steps=50).images

    # Convert to grid
    grid = torchvision.utils.make_grid([transforms.ToTensor()(img) for img in images], nrow=8)
    pil_grid = transforms.ToPILImage()(grid)
    pil_grid.save(f"{config.output_dir}/epoch_{epoch+1:04d}.png")

In [15]:
class KoopmanUNet(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        # 1. Time Embedding
        time_embed_dim = config.block_out_channels[0] * 4
        self.time_proj = Timesteps(config.block_out_channels[0], flip_sin_to_cos=True, downscale_freq_shift=0)
        self.time_embedding = TimestepEmbedding(config.block_out_channels[0], time_embed_dim)

        # 2. Pre-process
        self.conv_in = nn.Conv2d(config.in_channels, config.block_out_channels[0], kernel_size=3, padding=1)

        # 3. Down Blocks (Encoder)
        self.down_blocks = nn.ModuleList([])
        output_channel = config.block_out_channels[0]
        for i, down_block_type in enumerate(config.down_block_types):
            input_channel = output_channel
            output_channel = config.block_out_channels[i]
            is_final = i == len(config.down_block_types) - 1
            
            self.down_blocks.append(get_down_block(
                down_block_type, 
                num_layers=config.layers_per_block, 
                in_channels=input_channel,
                out_channels=output_channel, 
                temb_channels=time_embed_dim, 
                add_downsample=not is_final,
                resnet_eps=1e-5,
                resnet_act_fn="silu",
                resnet_groups=32, 
                attention_head_dim=8,
                downsample_padding=1
            ))

        # 4. Mid Block
        self.mid_block = UNetMidBlock2D(
            in_channels=config.block_out_channels[-1], 
            temb_channels=time_embed_dim,
            resnet_eps=1e-5,
            resnet_act_fn="silu",
            resnet_groups=32, 
            attention_head_dim=8,
            output_scale_factor=1
        )

        # 5. Koopman Bottleneck
        self.bottleneck_c = config.block_out_channels[-1]
        ds_factor = 2 ** (len(config.down_block_types) - 1)
        self.bottleneck_h = config.image_size // ds_factor
        self.bottleneck_w = config.image_size // ds_factor
        features = self.bottleneck_c * self.bottleneck_h * self.bottleneck_w
        
        self.koopman_operator = nn.Linear(features, features)
        print(f"Koopman Operator initialized with {features} features.")

        # 6. Up Blocks (Decoder)
        self.up_blocks = nn.ModuleList([])
        reversed_ch = list(reversed(config.block_out_channels))
        output_channel = reversed_ch[0]
        for i, up_block_type in enumerate(config.up_block_types):
            prev_output_channel = output_channel
            output_channel = reversed_ch[i]
            input_channel = reversed_ch[min(i + 1, len(config.block_out_channels) - 1)]
            is_final = i == len(config.up_block_types) - 1

            self.up_blocks.append(get_up_block(
                up_block_type, 
                num_layers=config.layers_per_block + 1, 
                in_channels=input_channel,
                out_channels=output_channel, 
                prev_output_channel=prev_output_channel,
                temb_channels=time_embed_dim, 
                add_upsample=not is_final,
                resnet_eps=1e-5,
                resnet_act_fn="silu",
                resnet_groups=32, 
                attention_head_dim=8
            ))
            prev_output_channel = output_channel

        # 7. Output
        self.conv_norm_out = nn.GroupNorm(32, config.block_out_channels[0], eps=1e-5)
        self.conv_act = nn.SiLU()
        self.conv_out = nn.Conv2d(config.block_out_channels[0], config.out_channels, kernel_size=3, padding=1)

    @property
    def device(self):
        """Helper for Diffusers pipeline compatibility."""
        return next(self.parameters()).device

    @property
    def dtype(self):
        """Helper for Diffusers pipeline compatibility."""
        return next(self.parameters()).dtype

    def forward(self, x, t, return_dict=False):
        # Handle Time
        t = t.to(x.device)
        if t.dim() == 0: t = t.unsqueeze(0).expand(x.shape[0])
        t_emb = self.time_embedding(self.time_proj(t))

        # Encoder
        x = self.conv_in(x)
        skips = (x,)
        for block in self.down_blocks:
            x, s = block(x, t_emb)
            skips += s
        
        # Mid & Koopman
        x = self.mid_block(x, t_emb)
        B, C, H, W = x.shape
        
        # Use reshape() instead of view() to handle non-contiguous memory
        x = self.koopman_operator(x.reshape(B, -1)).reshape(B, C, H, W)

        # Decoder
        for block in self.up_blocks:
            res_skips = skips[-len(block.resnets):]
            skips = skips[:-len(block.resnets)]
            x = block(x, res_skips, temb=t_emb)

        # Output
        x = self.conv_out(self.conv_act(self.conv_norm_out(x)))
        
        if return_dict: return {"sample": x}
        from diffusers.utils import BaseOutput
        return BaseOutput(sample=x)

In [16]:
def get_model(config, use_koopman=True):
    """
    Returns either the Custom KoopmanUNet or the Standard Diffusers UNet2DModel
    based on the boolean flag, ensuring identical architecture settings.
    """
    if use_koopman:
        print(f"Initializing Custom KoopmanUNet (Rank constrained bottleneck)...")
        return KoopmanUNet(config)

    else:
        print(f"Initializing Standard Baseline UNet2DModel...")
        return UNet2DModel(
            sample_size=config.image_size,
            in_channels=config.in_channels,
            out_channels=config.out_channels,
            layers_per_block=config.layers_per_block,
            block_out_channels=config.block_out_channels,
            down_block_types=config.down_block_types,
            up_block_types=config.up_block_types,
        )

In [17]:
def train(config):
    # get the correct model using the Toggle
    model = get_model(config, use_koopman=KOOPMAN).to(device)

    # standard Setup
    scheduler = DDPMScheduler(num_train_timesteps=1000)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
    dataloader = get_dataloader(config)
    lr_scheduler = get_cosine_schedule_with_warmup(
        optimizer, config.lr_warmup_steps, len(dataloader) * config.num_epochs
    )

    print(f"--- Starting Training: {config.output_dir} ---")

    for epoch in range(config.num_epochs):
        model.train()
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{config.num_epochs}")
        losses = []

        for x, _ in pbar:
            x = x.to(device)
            noise = torch.randn_like(x)
            t = torch.randint(0, 1000, (x.shape[0],), device=device).long()
            noisy_x = scheduler.add_noise(x, noise, t)

            # Diffusers UNet returns a tuple or object depending on return_dict
            # We unify this here:
            if KOOPMAN:
                noise_pred = model(noisy_x, t).sample
            else:
                noise_pred = model(noisy_x, t).sample

            loss = F.mse_loss(noise_pred, noise)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            lr_scheduler.step()

            losses.append(loss.item())
            pbar.set_postfix(loss=np.mean(losses[-100:]))

        # save images/models
        if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:
            save_sample(model, scheduler, epoch, config)

        if (epoch + 1) % config.save_model_epochs == 0:
            # handle saving differently for diffusers model vs ours
            save_path = f"{config.output_dir}/model.pth"
            if hasattr(model, "save_pretrained") and not KOOPMAN:
                 model.save_pretrained(config.output_dir) 
            else:
                 torch.save(model.state_dict(), save_path) 

    return model

In [26]:
model = train(config)

Initializing Custom KoopmanUNet (Rank constrained bottleneck)...
Koopman Operator initialized with 2048 features.
--- Starting Training: ddpm_mnist_koopman ---


Epoch 1/20:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 2/20:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 3/20:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 4/20:   0%|          | 0/938 [00:00<?, ?it/s]

...

Epoch 17/20:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 18/20:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 19/20:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 20/20:   0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [33]:
baseline_model = train(config)

Initializing Standard Baseline UNet2DModel...
--- Starting Training: ddpm_mnist_baseline ---


Epoch 1/20:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 2/20:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 3/20:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 4/20:   0%|          | 0/938 [00:00<?, ?it/s]

...

Epoch 17/20:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 18/20:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 19/20:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 20/20:   0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [18]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from mpl_toolkits.axes_grid1 import ImageGrid
import warnings

sns.set_theme(style="whitegrid", context="talk")
plt.rcParams['font.family'] = 'serif'

def get_latent_vectors(model, dataloader, device, limit_batches=20):
    """
    Extracts bottleneck vectors from a model for a subset of data.
    """
    model.eval()
    latents = []
    labels = []
    
    # Fixed timestep for consistent encoding
    t = torch.tensor([500], device=device).long()
    
    with torch.no_grad():
        for i, (x, y) in enumerate(dataloader):
            if i >= limit_batches: break
            x = x.to(device)
            
            # Handle Time Embedding
            t_batch = t.expand(x.shape[0])
            t_emb = model.time_embedding(model.time_proj(t_batch))
            
            # Encoder Pass
            h = model.conv_in(x)
            for block in model.down_blocks:
                h, _ = block(h, t_emb)
            h = model.mid_block(h, t_emb)
            
            # Flatten to latent vector
            vec = h.reshape(x.shape[0], -1)
            latents.append(vec.cpu().numpy())
            labels.append(y.numpy())
            
    return np.concatenate(latents), np.concatenate(labels)

def plot_spectral_analysis(model, output_dir="."):
    """
    Generates Singular Value and Eigenvalue plots for the Koopman Operator.
    Saves: koopman_singular_values.png, koopman_eigenvalues.png
    """
    print("--- Generating Spectral Analysis ---")
    model.eval()
    
    # Extract the matrix K
    # Assumes model.koopman_operator is an nn.Linear layer
    K = model.koopman_operator.weight.detach().cpu().numpy()
    
    # 1. Singular Value Decomposition (Rank Analysis)
    U, S, Vh = np.linalg.svd(K)
    
    plt.figure(figsize=(8, 5))
    plt.plot(S, 'o-', color='#1f77b4', markersize=4, linewidth=1.5)
    plt.yscale('log')
    plt.title(r"Singular Values of Operator $K$", fontsize=16)
    plt.xlabel("Index")
    plt.ylabel(r"Singular Value $\sigma$ (Log Scale)")
    plt.grid(True, which="both", ls="--", alpha=0.6)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/koopman_singular_values.png", dpi=300)
    plt.close()
    
    # 2. Eigenvalue Spectrum (Stability Analysis)
    eigenvalues, eigenvectors = np.linalg.eig(K)
    
    plt.figure(figsize=(8, 8))
    ax = plt.gca()
    
    # Unit Circle
    circle = plt.Circle((0, 0), 1, color='black', fill=False, linestyle='--', linewidth=2, label="Unit Circle")
    ax.add_patch(circle)
    
    # Plot Eigenvalues
    plt.scatter(eigenvalues.real, eigenvalues.imag, alpha=0.6, color='#d62728', s=30)
    
    plt.title(r"Eigenvalues $\lambda$ in Complex Plane", fontsize=16)
    plt.xlabel("Real Part")
    plt.ylabel("Imaginary Part")
    plt.axis('equal')
    plt.legend(loc='upper right')
    plt.grid(True, ls=":", alpha=0.5)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/koopman_eigenvalues.png", dpi=300)
    plt.close()
    
    return eigenvalues, eigenvectors

def plot_koopman_modes(model, config, device, eigenvalues, eigenvectors, output_dir="."):
    """
    Visualizes the top Koopman Modes by decoding eigenvectors.
    Saves: koopman_modes.png
    """
    print("--- Generating Koopman Modes ---")
    model.eval()
    
    # 1. Get Dummy Skip Connections
    # The decoder needs skip connections. We generate "background" skips 
    # by passing a zero tensor through the encoder.
    dummy_input = torch.zeros(1, config.in_channels, config.image_size, config.image_size).to(device)
    t = torch.tensor([500], device=device).long()
    t_emb = model.time_embedding(model.time_proj(t))
    
    dummy_skips = []
    with torch.no_grad():
        x = model.conv_in(dummy_input)
        dummy_skips.append(x)
        for block in model.down_blocks:
            x, skips = block(x, t_emb)
            dummy_skips.extend(skips)
    
    # 2. Sort Eigenvectors by Magnitude
    sorted_indices = np.argsort(np.abs(eigenvalues))[::-1]
    
    # 3. Decode Top Modes
    num_modes = 8
    modes_to_plot = []
    
    for i in range(num_modes):
        idx = sorted_indices[i]
        vec = eigenvectors[:, idx].real # Take real part for visualization
        
        # Prepare latent tensor
        z = torch.from_numpy(vec).float().to(device)
        z = z.reshape(1, model.bottleneck_c, model.bottleneck_h, model.bottleneck_w)
        
        # Decode
        with torch.no_grad():
            # We must copy the skips list because .pop() is destructive
            current_skips = [s.clone() for s in dummy_skips]
            
            x_dec = z
            for block in model.up_blocks:
                # Get correct number of skips for this block (usually 2 or 3)
                num_resnets = len(block.resnets)
                skips_for_block = current_skips[-num_resnets:]
                current_skips = current_skips[:-num_resnets]
                
                x_dec = block(x_dec, tuple(skips_for_block), temb=t_emb)
            
            out = model.conv_out(model.conv_act(model.conv_norm_out(x_dec)))
            
        # Normalize for display
        img = out.squeeze().cpu().numpy()
        img = (img - img.min()) / (img.max() - img.min())
        modes_to_plot.append(img)

    # 4. Plot Grid
    fig = plt.figure(figsize=(16, 3))
    grid = ImageGrid(fig, 111, nrows_ncols=(1, num_modes), axes_pad=0.1)
    
    for ax, img, i in zip(grid, modes_to_plot, range(num_modes)):
        ax.imshow(img, cmap='viridis') # Viridis is good for abstract modes
        ax.set_title(f"Mode {i+1}\n$|\lambda|={np.abs(eigenvalues[sorted_indices[i]]):.2f}$", fontsize=10)
        ax.axis('off')
        
    plt.savefig(f"{output_dir}/koopman_modes.png", dpi=300)
    plt.close()

def plot_latent_comparison(koopman_model, baseline_model, dataloader, device, output_dir="."):
    """
    Comparing latent spaces using t-SNE.
    Saves: koopman_tsne_comparison.png
    """
    print("--- Generating Latent Space Comparison ---")
    
    # 1. Get Latents
    print("Extracting Koopman latents...")
    k_vecs, k_labels = get_latent_vectors(koopman_model, dataloader, device)
    
    print("Extracting Baseline latents...")
    # Note: baseline_model must be on device
    baseline_model.to(device)
    b_vecs, b_labels = get_latent_vectors(baseline_model, dataloader, device)
    
    # 2. PCA (Pre-reduction for speed/stability)
    pca = PCA(n_components=50)
    k_pca = pca.fit_transform(k_vecs)
    b_pca = pca.fit_transform(b_vecs)
    
    # 3. t-SNE
    print("Running t-SNE...")
    tsne = TSNE(n_components=2, perplexity=30, random_state=42)
    k_tsne = tsne.fit_transform(k_pca)
    b_tsne = tsne.fit_transform(b_pca)
    
    # 4. Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
    
    # Koopman Plot
    sns.scatterplot(x=k_tsne[:,0], y=k_tsne[:,1], hue=k_labels, palette="tab10", ax=ax1, s=50, alpha=0.7, legend=False)
    ax1.set_title("Koopman U-Net Latent Space", fontsize=16)
    ax1.axis('off')
    
    # Baseline Plot
    sns.scatterplot(x=b_tsne[:,0], y=b_tsne[:,1], hue=b_labels, palette="tab10", ax=ax2, s=50, alpha=0.7)
    ax2.set_title("Standard U-Net Latent Space", fontsize=16)
    ax2.axis('off')
    ax2.legend(title="Digit Class", bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/koopman_tsne_comparison.png", dpi=300)
    plt.close()

def run_full_dashboard(koopman_model, baseline_model, dataloader, config, device):
    """Orchestrates the full analysis suite."""
    # 1. Spectral
    evals, evecs = plot_spectral_analysis(koopman_model)
    
    # 2. Modes
    plot_koopman_modes(koopman_model, config, device, evals, evecs)
    
    # 3. Comparison 
    if baseline_model is not None:

        plot_latent_comparison(koopman_model, baseline_model, dataloader, device)

<>:160: SyntaxWarning: invalid escape sequence '\l'
<>:160: SyntaxWarning: invalid escape sequence '\l'
/var/folders/7q/d3z58krn1slbn5fckpc1y7jm0000gn/T/ipykernel_25192/1603423301.py:160: SyntaxWarning: invalid escape sequence '\l'
  ax.set_title(f"Mode {i+1}\n$|\lambda|={np.abs(eigenvalues[sorted_indices[i]]):.2f}$", fontsize=10)


In [22]:
# Loading Koopman Model
model = KoopmanUNet(config)
state_dict = torch.load("ddpm_mnist_koopman/model.pth", map_location=device)
model.load_state_dict(state_dict)
model.to(device)
model.eval()

# Loading Baseline
baseline_model = UNet2DModel.from_pretrained("ddpm_mnist_baseline/")
baseline_model.to(device)
baseline_model.eval()

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Koopman Operator initialized with 2048 features.


UNet2DModel(
  (conv_in): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=32, out_features=128, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=128, out_features=128, bias=True)
  )
  (down_blocks): ModuleList(
    (0): DownBlock2D(
      (resnets): ModuleList(
        (0-1): 2 x ResnetBlock2D(
          (norm1): GroupNorm(32, 32, eps=1e-05, affine=True)
          (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (time_emb_proj): Linear(in_features=128, out_features=32, bias=True)
          (norm2): GroupNorm(32, 32, eps=1e-05, affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (nonlinearity): SiLU()
        )
      )
      (downsamplers): ModuleList(
        (0): Downsample2D(
          (conv): Conv2d(32, 32, ker

In [23]:
train_dataloader = get_dataloader(config)
run_full_dashboard(model, baseline_model, train_dataloader, config, device)

--- Generating Spectral Analysis ---
--- Generating Koopman Modes ---
--- Generating Latent Space Comparison ---
Extracting Koopman latents...
Extracting Baseline latents...
Running t-SNE...
